[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepmipt/RDLS_NLP_2022/blob/main/lectures/04.Creating_Dialog_Systems/seminar/notebooks/03.responses.ipynb)

In [ ]:
# !pip install df_engine df_runner df_db_connector df_telegram_connector

<img src="https://raw.githubusercontent.com/kudep/RDLS_images/bba12c082827415f7d3e1ab09dfb94d68415be93/dfe_examples/script_structire.svg" >
<img src="https://raw.githubusercontent.com/kudep/RDLS_images/bba12c082827415f7d3e1ab09dfb94d68415be93/dfe_examples/node_sructire.svg" >

## Short description
Here we will consider different options for setting responses.

The response is set by any object of python:

- collable objects - If the object is callable, then it must have a special signature:
                     func`(ctx: Context, actor: Actor, *args, **kwargs) -> Any`
                     Then this object will be called with the appropriate arguments.

- non-callable objects - If the object is not callable,
                         then the object will be returned by the agent as a response.


Out of the box, df_engine offers 1 additional response function:
- `choice` - will return `true` if the user's request completely matches the value passed to the function.

In [2]:
from df_engine.core import Actor, Context
from typing import Any

def cannot_talk_about_topic_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    request = ctx.last_request
    topic_pattern = re.compile(r"(.*talk about )(.*)\.")
    topic = topic_pattern.findall(request)
    topic = topic and topic[0] and topic[0][-1]
    if topic:
        return f"Sorry, I can not talk about {topic} now."
    else:
        return "Sorry, I can not talk about that now."


def upper_case_response(response: str):
    # wrapper for internal response function
    def cannot_talk_about_topic_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
        return response.upper()

    return cannot_talk_about_topic_response


def fallback_trace_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    return {"previous_node": list(ctx.labels.values())[-2], "last_request": ctx.last_request}

In [3]:
# imports
from df_engine.core.keywords import TRANSITIONS, RESPONSE
import df_engine.conditions as cnd
import df_engine.responses as rsp
import re
script = {
    "greeting_flow": {
        "start_node": {  # This is an initial node, it doesn't need an `RESPONSE`
            RESPONSE: "",
            TRANSITIONS: {"node1": cnd.exact_match("Hi")},  # If "Hi" == request of user then we make the transition
        },
        "node1": {
            RESPONSE: rsp.choice(["Hi, what is up?", "Hello, how are you?"]),  # random choice from candicate list
            TRANSITIONS: {"node2": cnd.exact_match("i'm fine, how are you?")},
        },
        "node2": {
            RESPONSE: "Good. What do you want to talk about?",
            TRANSITIONS: {"node3": cnd.exact_match("Let's talk about music.")},
        },
        "node3": {RESPONSE: cannot_talk_about_topic_response, TRANSITIONS: {"node4": cnd.exact_match("Ok, goodbye.")}},
        "node4": {RESPONSE: upper_case_response("bye"), TRANSITIONS: {"node1": cnd.exact_match("Hi")}},
        "fallback_node": {  # We get to this node if an error occurred while the agent was running
            RESPONSE: fallback_trace_response,
            TRANSITIONS: {"node1": cnd.exact_match("Hi")},
        },
    }
}

In [4]:
from df_runner import ScriptRunner

runner = ScriptRunner(
    script,
    start_label=("greeting_flow", "start_node"),
    fallback_label=("greeting_flow", "fallback_node"),
)

In [ ]:
runner.start()